In [1]:
import numpy as np
import pandas as pd
from hepunits import constants as c
from hepunits import units as u
from matplotlib import pyplot as plt

In [2]:
# Get decay rates and daughters of isotopes

import re
import urllib.request
from functools import lru_cache


@lru_cache(maxsize=None)
def lc_pd_dataframe(**kwargs):
    url = "https://nds.iaea.org/relnsd/v1/data?"
    url += "&".join((f"{k}={v}" for k, v in kwargs.items()))
    req = urllib.request.Request(url)
    req.add_header(
        "User-Agent",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0",
    )
    return pd.read_csv(urllib.request.urlopen(req))

In [3]:
all_ground_states = lc_pd_dataframe(fields="ground_states", nuclides="all")


@lru_cache(maxsize=None)
def get_decay_rates(nuclide):
    A, symbol = re.fullmatch("(\d+)(\S+)", nuclide).groups()
    A = int(A)
    ground_state = all_ground_states.query(f"n + z == {A} & symbol == '{symbol}'").iloc[
        0
    ]
    half_life = ground_state[f"half_life_sec"] * u.s
    if np.isnan(half_life):
        half_life = 1e-99 * u.s
    Z = ground_state["z"]
    N = ground_state["n"]
    daughters = {}
    total_ratio = 0
    for i_d in range(1, 4):
        decay_mode = ground_state[f"decay_{i_d}"]
        if not isinstance(decay_mode, str):
            break
        decay_ratio = ground_state[f"decay_{i_d}_%"]
        if np.isnan(decay_ratio):
            decay_ratio = 1e-99

        if "B-" in decay_mode:
            d_n = N - 1
            d_z = Z + 1
        elif "B+" in decay_mode or "EC" in decay_mode:
            d_n = N + 1
            d_z = Z - 1
        elif "A" in decay_mode:
            d_n = N - 2
            d_z = Z - 2
        elif "N" in decay_mode:
            d_n = N - 1
            d_z = Z
        elif "P" in decay_mode:
            d_n = N
            d_z = Z - 1
        else:
            break

        d_symbol = all_ground_states.query(f"n == {d_n} & z == {d_z}").iloc[0]["symbol"]
        decay = {"d_symbol": d_symbol, "d_n": d_n, "d_z": d_z}

        dsym = decay["d_symbol"]
        dA = decay["d_n"] + decay["d_z"]
        daughters[f"{dA}{dsym}"] = decay_ratio
        total_ratio += decay_ratio

    for k in daughters:
        # Normalise branching ratios
        daughters[k] /= total_ratio
        # Multiply with decay rate
        daughters[k] *= np.log(2) / half_life

    return daughters

In [4]:
get_decay_rates("180Ta")

{'180Hf': 2.007110019199689e-14, '180W': 3.5419588574112156e-15}

In [5]:
levels = lc_pd_dataframe(fields="levels", nuclides="180Ta")

In [6]:
levels.columns

Index(['z', 'n', 'symbol', 'idx', 'energy_shift', 'energy', 'unc_e',
       'ripl_shift', 'jp', 'jp_order', 'half_life', 'operator_hl', 'unc_hl',
       'unit_hl', 'half_life_sec', 'unc_hl.1', 'decay_1', 'decay_1_%', 'unc_1',
       'decay_2', 'decay_2_%', 'unc_2', 'decay_3', 'decay_3_%', 'unc_3',
       'isospin', 'magnetic_dipole', 'unc_mn', 'electric_quadrupole', 'unc_eq',
       'ENSDF_publication_cut-off', 'ENSDF_authors', 'Extraction_date'],
      dtype='object')

In [7]:
levels[["z", "n", "symbol", "half_life_sec", "energy"]]

,z,n,symbol,half_life_sec,energy
0,73,107,Ta,2.935440e+04,0.00
1,73,107,Ta,NaN,39.54
2,73,107,Ta,2.240542e+23,77.20
3,73,107,Ta,1.920000e-08,107.78
4,73,107,Ta,NaN,110.75
...,...,...,...,...,...
235,73,107,Ta,NaN,3832.20
236,73,107,Ta,NaN,3855.50
237,73,107,Ta,NaN,4170.90
238,73,107,Ta,1.700000e-05,4169.90


In [8]:
states = lc_pd_dataframe(fields="ground_states", nuclides="all")

In [9]:
states.columns

Index(['z', 'n', 'symbol', 'radius', 'unc_r', 'abundance', 'unc_a',
       'energy_shift', 'energy', 'unc_e', 'ripl_shift', 'jp', 'half_life',
       'operator_hl', 'unc_hl', 'unit_hl', 'half_life_sec', 'unc_hls',
       'decay_1', 'decay_1_%', 'unc_1', 'decay_2', 'decay_2_%', 'unc_2',
       'decay_3', 'decay_3_%', 'unc_3', 'isospin', 'magnetic_dipole', 'unc_md',
       'electric_quadrupole', 'unc_eq', 'qbm', 'unc_qb', 'qbm_n', 'unc_qbmn',
       'qa', 'unc_qa', 'qec', 'unc_qec', 'sn', 'unc_sn', 'sp', 'unc_sp',
       'binding', 'unc_ba', 'atomic_mass', 'unc_am', 'massexcess', 'unc_me',
       'me_systematics', 'discovery', 'ENSDFpublicationcut-off',
       'ENSDFauthors', 'Extraction_date'],
      dtype='object')

In [10]:
states[
    [
        "symbol",
        "z",
        "n",
        "abundance",
        "energy",
        "half_life_sec",
        "decay_1",
        "decay_1_%",
        "decay_2",
        "decay_2_%",
        "decay_3",
        "decay_3_%",
    ]
]

,symbol,z,n,abundance,energy,half_life_sec,decay_1,decay_1_%,decay_2,decay_2_%,decay_3,decay_3_%
0,n,0,1,NaN,0.0,6.139000e+02,B-,100.0,NaN,NaN,NaN,NaN
1,n,0,4,NaN,0.0,1.754760e-22,NaN,NaN,NaN,NaN,NaN,NaN
2,n,0,6,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H,1,0,99.9855,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H,1,1,0.0145,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3381,Lv,116,177,NaN,0.0,9.500000e-02,A,100.0,SF,12.5,NaN,NaN
3382,Lv,116,178,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3383,Ts,117,176,NaN,0.0,2.100000e-02,A,100.0,SF,6.0,NaN,NaN
3384,Ts,117,177,NaN,0.0,5.100000e-02,A,100.0,NaN,NaN,NaN,NaN


In [11]:
states["decay_1"].unique()

array(['B-', nan, 'N', '2N', 'P', 'B-A', 'A', 'EC', 'EC+B+', '2P', 'B+P',
       'B+', 'B-N', 'B-2N', '2B-', 'ECP', '2EC', '2B+', 'IT', 'SF',
       'ECSF'], dtype=object)

In [12]:
states[states["decay_1"] == "P"][
    [
        "symbol",
        "z",
        "n",
        "abundance",
        "energy",
        "half_life_sec",
        "decay_1",
        "decay_1_%",
        "decay_2",
        "decay_2_%",
        "decay_3",
        "decay_3_%",
    ]
]

,symbol,z,n,abundance,energy,half_life_sec,decay_1,decay_1_%,decay_2,decay_2_%,decay_3,decay_3_%
18,Li,3,1,NaN,0.0,NaN,P,100.0,NaN,NaN,NaN,NaN
19,Li,3,2,NaN,0.0,3.709250e-22,P,NaN,NaN,NaN,NaN,NaN
38,B,5,2,NaN,0.0,5.695852e-22,P,100.0,NaN,NaN,NaN,NaN
40,B,5,4,NaN,0.0,8.448847e-19,P,100.0,NaN,NaN,NaN,NaN
67,N,7,3,NaN,0.0,1.824951e-22,P,100.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2435,Ir,77,88,NaN,0.0,1.000000e-06,P,NaN,A,NaN,NaN,NaN
2520,Au,79,91,NaN,0.0,2.900000e-04,P,89.0,A,11.0,NaN,NaN
2521,Au,79,92,NaN,0.0,2.200000e-05,P,100.0,NaN,NaN,NaN,NaN
2608,Tl,81,95,NaN,0.0,5.200000e-03,P,100.0,NaN,NaN,NaN,NaN


In [13]:
states[states["qbm"] <= 0].sort_values("qbm")[["symbol", "z", "n", "qbm"]]

,symbol,z,n,qbm
27,Be,4,2,-28945.0000
19,Li,3,2,-25460.0000
251,S,16,12,-24197.0000
297,Ar,18,14,-24190.0000
85,O,8,6,-23956.6215
...,...,...,...,...
1901,Eu,63,85,-28.0630
2544,Au,79,115,-27.9978
3117,Am,95,148,-6.9302
2844,Rn,86,136,-6.1461


In [26]:
nuc = "13C"

In [27]:
alpha = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="a")

In [28]:
beta = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="bm")

In [29]:
betaplus = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="bp")

In [30]:
binbeta = lc_pd_dataframe(fields="bin_beta", nuclides=nuc, rad_types="bm")

In [31]:
gamma = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="g")

In [32]:
alpha.columns

Index(['0'], dtype='object')

In [33]:
beta.columns

Index(['0'], dtype='object')

In [34]:
betaplus.columns

Index(['0'], dtype='object')

In [35]:
betaplus

,0


In [36]:
gamma.columns

Index(['0'], dtype='object')

In [37]:
gamma[["energy", "intensity", "decay", "p_energy", "p_symbol", "d_symbol"]]

KeyError: "None of [Index(['energy', 'intensity', 'decay', 'p_energy', 'p_symbol', 'd_symbol'], dtype='object')] are in the [columns]"

In [38]:
binbeta.columns

Index(['p_z', 'p_n', 'p_symbol', 'p_energy', 'd_z', 'd_n', 'd_symbol',
       'bin_en', 'dn_de', 'unc_dn_de', 'dn_de_nu', 'unc_dn_de_nu',
       'extraction_date'],
      dtype='object')

In [39]:
binbeta

,p_z,p_n,p_symbol,p_energy,d_z,d_n,d_symbol,bin_en,dn_de,unc_dn_de,dn_de_nu,unc_dn_de_nu,extraction_date


In [40]:
heaviest_ground_states = (
    all_ground_states.query("abundance > 0")
    .sort_values("n", ascending=False)
    .drop_duplicates("z")
)

In [41]:
for heav in heaviest_ground_states.itertuples():
    z = heav.z - 1
    n = heav.n + 1
    try:
        signal = all_ground_states.query(f"z == {z} & n == {n}").iloc[0]
    except IndexError:
        continue

    if signal.half_life_sec < 60 * 60:
        continue

    nuc = f"{signal.n + signal.z}{signal.symbol}"

    gammas = lc_pd_dataframe(fields="decay_rads", nuclides=nuc, rad_types="g")
    try:
        maxg = gammas.intensity.max()
    except AttributeError:
        continue

    if maxg < 10:
        continue

    print(nuc, maxg)

231Th 56.0
197Pt 45.8869353095
187W 33.2
181Hf 80.5
176Yb 93.0
165Dy 14.24542455
159Gd 35.7
153Sm 50.1
142La 47.4
141Ce 48.3
139Ba 23.7249
133Xe 36.9
127Te 18.869303630720506
123Sn 85.69
115Cd 46.02
109Pd 56.0
103Ru 91.0
87Kr 49.6
75Ge 39.5
65Ni 23.59
59Fe 56.5
41Ar 99.16


In [42]:
# 12C -> 12N nucleon knockout threshold, i.e. 12C -> 11C
E_B12C = 12 * 7680 * u.keV
E_B11C = 11 * 6676 * u.keV
E_t = E_B12C - E_B11C / u.MeV
E_t

18.72399999999999

In [43]:
# Very rough, assume M >> p_l
# E_t = E_max - M = sqrt((2 * p_l)^2 + M^2) - M
# (E_t + M)^2 = (2 * p_l)^2 + M^2
M = 0.938 * u.GeV
p_l = 1 / 2 * np.sqrt(E_t**2 + 2 * E_t * M)
p_l

94.1764463334649

In [103]:
# Nuclides with lowest threshold for CC interaction
stableish = pd.DataFrame(
    all_ground_states.query(
        "abundance > 1"
    )
)

In [120]:
stableish.query("qec > -450 & -qec < sn & -qec < sp").sort_values("qec")[
    ["z", "n", "symbol", "abundance", "atomic_mass", "qec", "sn", "sp", "half_life_sec"]
].iloc[:20]

,z,n,symbol,abundance,atomic_mass,qec,sn,sp,half_life_sec
2376,75,110,Re,37.4000,1.849530e+08,-431.176,7670.5462,5402.5658,NaN
1590,55,78,Cs,100.0000,1.329055e+08,-427.360,8989.5679,6080.9396,NaN
1422,51,70,Sb,57.2100,1.209038e+08,-402.531,9253.3416,5790.3870,NaN
3009,91,140,Pa,100.0000,2.310359e+08,-391.473,6820.5232,4727.1449,1.027809e+12
575,28,30,Ni,68.0769,5.793534e+07,-381.579,12216.2285,8172.1845,NaN
1049,42,50,Mo,14.6490,9.190681e+07,-355.297,12671.0710,7459.5306,NaN
2140,69,100,Tm,100.0000,1.689342e+08,-353.491,8033.5936,5574.3940,NaN
1337,49,64,In,4.2810,1.129041e+08,-323.837,9448.3147,6081.2372,NaN
2461,77,114,Ir,37.3000,1.909606e+08,-313.587,8026.5301,5289.9723,NaN
1129,44,55,Ru,12.7600,9.890593e+07,-297.516,7471.8315,8482.1420,NaN
